__WHAT QUESTION DO I WANT TO ANSWER??__

# Learning One Hot Encoding in Q
First we take a peek into the date and decide on the datatype 

In [ ]:
\c 200 200
headers:"," vs raze 1#read0 `:Datasets/vgsales.csv
data:"," vs raze 1_2#read0 `:Datasets/vgsales.csv
headers!data
/rank is id - J
/Name is unique - C
/Platform is repeated string - S
/Year is year - D
/Genre is repeated string - S
/Publisher is repeated string -S
/NA_Sales - F
/EU_Sales - F
/JP_Sales - F
/Other_Sales - F
/Global_Sales - F

10#vgsales:("J*S*SSFFFFF";enlist ",") 0: `:Datasets/vgsales.csv

## Enumeration as Categorical Encoding
In kdb q, we are able to enumerate the symbols and this allows us to jump between int and sym category

In [44]:
/Here we will enumerate platform, genre, and publisher
2#distPlatform:exec distinct Platform from vgsales
2#distGenre:exec distinct Genre from vgsales
2#distPublisher: exec distinct Publisher from vgsales
5#vgEumsales:update `distPlatform $ Platform, `distGenre$Genre, `distPublisher$Publisher from vgsales

/
The above will enumerate the sym (category) into specific placeholder such as distPlatform
This means that we are able to use "i"${item} to cast it back to category number
Here we try to change them into int before creating the one hot encoding columns
\
5#vgTmpSale:update "i"$Platform,"i"$Genre,"i"$Publisher from vgEumsales


`Wii`NES


`Sports`Platform


`Nintendo`Microsoft Game Studios


Rank Name                       Platform Year   Genre        Publisher NA_Sales EU_Sales JP_Sales Other_Sales Global_Sales
--------------------------------------------------------------------------------------------------------------------------
1    "Wii Sports"               Wii      "2006" Sports       Nintendo  41.49    29.02    3.77     8.46        82.74       
2    "Super Mario Bros."        NES      "1985" Platform     Nintendo  29.08    3.58     6.81     0.77        40.24       
3    "Mario Kart Wii"           Wii      "2008" Racing       Nintendo  15.85    12.88    3.79     3.31        35.82       
4    "Wii Sports Resort"        Wii      "2009" Sports       Nintendo  15.75    11.01    3.28     2.96        33          
5    "Pokemon Red/Pokemon Blue" GB       "1996" Role-Playing Nintendo  11.27    8.89     10.22    1           31.37       


Rank Name                       Platform Year   Genre Publisher NA_Sales EU_Sales JP_Sales Other_Sales Global_Sales
-------------------------------------------------------------------------------------------------------------------
1    "Wii Sports"               0        "2006" 0     0         41.49    29.02    3.77     8.46        82.74       
2    "Super Mario Bros."        1        "1985" 1     0         29.08    3.58     6.81     0.77        40.24       
3    "Mario Kart Wii"           0        "2008" 2     0         15.85    12.88    3.79     3.31        35.82       
4    "Wii Sports Resort"        0        "2009" 0     0         15.75    11.01    3.28     2.96        33          
5    "Pokemon Red/Pokemon Blue" 2        "1996" 3     0         11.27    8.89     10.22    1           31.37       


## Categorical Encoding with One Hot
From the enumerated table, we seek to create additional columns that represents each category in kdb Q
To do so, we make use of dictionary which is mroe flexible than tables, but this can be done with q-sql syntax as well
<br> Here we use the flexibility of q to synthetically produce new columns which contains the one hot encoding
<br> This will not be the most efficient way if there are too many categories, if thats the case, we might want to use hash featuring
> Hash featuring, might be considered in the future 

In [87]:
/Here we use the flexibility of q to synthetically produce new columns which contains the one hot encoding
/This will not be the most efficient way if there are too many categories, if thats the case, we might want to use hash featuring
/Hash featuring, might be considered in the future especially for Publisher which has 579

.encode.createNewColumns:{[enumtable;column] `$string[column],/:"_" ,/: string each distinct enumtable[column]}
.encode.createNewData:{[enumtable;column] enumtable[column] =/: distinct enumtable[column]}

enlist[`newPlatformColumns]!enlist count newPlatformColumns:.encode.createNewColumns[vgTmpSale;`Platform]
newPlatformData:.encode.createNewData[vgTmpSale;`Platform]

enlist[`newGenreColumns]!enlist count newGenreColumns:.encode.createNewColumns[vgTmpSale;`Genre]
newGenreData:.encode.createNewData[vgTmpSale;`Genre]

enlist[`newPublisherColumns]!enlist count newPublisherColumns:.encode.createNewColumns[vgTmpSale;`Publisher]
newPublisherData:.encode.createNewData[vgTmpSale;`Publisher]

5#-5#'vgSaleFinal:vgTmpSale,'flip[newPlatformColumns!newPlatformData],'flip[newGenreColumns!newGenreData],'flip[newPublisherColumns!newPublisherData]

/A look at the final table columns
cols vgSaleFinal
count cols vgSaleFinal

newPlatformColumns| 31


newGenreColumns| 12


newPublisherColumns| 579


Publisher_574 Publisher_575 Publisher_576 Publisher_577 Publisher_578
---------------------------------------------------------------------
0             0             0             0             0            
0             0             0             0             0            
0             0             0             0             0            
0             0             0             0             0            
0             0             0             0             0            


`Rank`Name`Platform`Year`Genre`Publisher`NA_Sales`EU_Sales`JP_Sales`Other_Sales`Global_Sales`Platform_0`Platform_1`Platform_2`Platform_3`Platform_4`Platform_5`Platform_6`Platform_7`Platform_8`Platf..


633


## Hash Featuring
With Hash Featuring we can compress hundred of columns into a small bunch, minimizing computational needs
<br> This utilises sklearn python library which makes this complicated problem into a simple liner
<br> If we use this function, we do not have to enumerate the data and this could be done from the on start. 
<br> Both has its pros and cons

In [127]:
featureHashing:{[nFeature;inputType;data;colName]
    FeatureHasher:.p.import[`sklearn.feature_extraction][`:FeatureHasher];
    fh:FeatureHasher[`n_features pykw nFeature;`input_type pykw inputType];
    hashed_features:fh[`:fit_transform] data;
    hashed_features:hashed_features[`:toarray][];
    data:hashed_features`;
    colName:`$,["fh";colName],/:"_",/: string til count first `float$data;
    flip colName!flip data
    }

10#flip featureHashing[30;"string";vgsales `Publisher;"Publisher"]

/
from sklearn.feature_extraction import FeatureHasher
fh = FeatureHasher(n_features=6, input_type='string')
hashed_features = fh.fit_transform(vg_df['Genre'])
hashed_features = hashed_features.toarray()
pd.concat([vg_df[['Name', 'Genre']], pd.DataFrame(hashed_features)], 
          axis=1).iloc[1:7]
\


fhPublisher_0| 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ..
fhPublisher_1| 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  -3 -3 0  0  0  0  0  -3 -3 0  0  0  0  0  0  0  0  0  0  0  0  0  -3 0  0  0  0  0  -3 0  0  0  0  0  0  -3 0  0  0  0  -3 0  0  0  0 ..
fhPublisher_2| 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  1  1  0  0  0  0  0  1  1  0  0  0  2  0  0  0  0  0  0  0  0  0  1  0  0  0  0  2  1  0  0  2  0  0  0  1  2  0  2  0  1  0  0  0  0 ..
fhPublisher_3| 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ..
fhPublisher_4| 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  1  0  1  0  0  0  0  0  0 ..


## Prediction of Sales from Genre and Publisher
Is there predictive power from genre or publisher for potential sales generated ?
- Correlation of Genre against sales
- Correlation of publisher against sales
- Is there a need for cross encoding ?
- Is there a need to keep both genre and publisher (correlation test, high correlation means not needed)

### Correlation Test
By correlating datasets, we are able to tell if we can remove overlapping data 

In [88]:
correlationColumns:`Genre,newPlatformColumns
correlationColumns:?[vgSaleFinal;();0b;correlationColumns!correlationColumns]

10#flip correlationColumns

Genre     | 0 1 2 0 3 4 1 5 1 6 7 2 3 0 0 5 8 8 1 5 3 1 1 8 8 3 3 4 2 6 3 6 3 6 6 6 6 6 8 9 6 7 2 6 8 8 1 2 1 3 10 8 2 1 2 6 8 1 3 1 5 6 6 2 1 6 3 6 5 2 6 1 5 7 5 3 2 0 5 6 5 3 8 7 6 0 3 3 3 4 8 1 ..
Platform_0| 1 0 1 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0  0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 ..
Platform_1| 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ..
Platform_2| 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ..
Platform_3| 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0  0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 ..


In [90]:
/Correlation matrix
show t:([]a:5?1.0;b:5?1.0;c:5?1.0)
u cor/:\:u:flip t

a         b         c         
------------------------------
0.726781  0.6708738 0.7520102 
0.4046546 0.6789082 0.1086824 
0.8355065 0.412317  0.9598964 
0.642737  0.9877844 0.03668341
0.5830262 0.3867353 0.6430982 


 | a          b          c         
-| --------------------------------
a| 1          -0.2262263 0.739021  
b| -0.2262263 1          -0.7780611
c| 0.739021   -0.7780611 1         


## Top Sales Publisher vs Big Publisher


In [ ]:
\dir